In [1]:
%cd /home/q123/Desktop/explo

import torch 
import gpytorch 
import logging
import logging.config

from src.helpers import setup_experiment
from src.trainer import Trainer
from src.optimizers.gibo import GIBOptimizer
from src.optimizers.vanilla import BOptimizer
from src.config import get_configs

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_name = "CartPole-v1"
kernel_name = "rbf" ## "linearstate" /"rbfstate"

env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
additional_layers=[] ### can be empty or [8,7] for adding 2 layers with width 8,7 respectively
model,objective_env = setup_experiment(env_config,kernel_config,likelihood_config,additional_layers)

trainer_config = {
        "n_steps" :200,
        "report_freq":10,
        "save_best":False,
}

### Chose optimizer 
#optimizer = BOptimizer(**optimizer_config)
optimizer = GIBOptimizer(model,**optimizer_config)
trainer = Trainer(model,objective_env,optimizer,**trainer_config)
rslt= trainer.run()

MathLog.src.helpers : WARNING : MLP dimensions : [4, 1]
Using ard_num_dims = 5
 Gibo will use 20 last points to fit GP and 8 info samples
current 0.4500005543231964 / max 0.9999942183494568 /batch_mean 0.28920021653175354 /batch_max 0.5400001406669617 
##############################
covar_lengthscale max 0.3203306496143341 / min 0.3203306496143341                      covar_outputscale 0.010000034235417843                     noise 0.20000006258487701
##############################
last parameters tensor([-0.0349,  0.1669, -0.0599,  0.5277, -0.0721])
MLL : -0.3369414210319519
current 0.7619972825050354 / max 0.9999942183494568 /batch_mean 0.887995719909668 /batch_max 0.9999942183494568 
##############################
covar_lengthscale max 0.29277071356773376 / min 0.29277071356773376                      covar_outputscale 0.010000034235417843                     noise 0.20000006258487701
##############################
last parameters tensor([0.1404, 0.3328, 0.3607, 0.4391, 0.0841])
MLL

KeyboardInterrupt: 